In [1]:
import joblib
import pandas as pd
q_a_dict_path="develop/q_a_dict.bin"
text_dict=joblib.load(q_a_dict_path)
qa_df=pd.read_csv("develop/q_and_a_test.csv")


In [2]:
q_to_a_dict={}
for i in range(len(qa_df)):
    q=qa_df["Q"][i]
    a=qa_df["A"][i]
    q_to_a_dict[q]=a


In [17]:
import random

q_list=list(qa_df["Q"])
a_list=list(qa_df["A"])

q_a_dataset=[]
for q in q_list:
    q_a_dataset.append((q,q_to_a_dict[q],1))


for i in range(len(q_list)*10):
    q=random.choice(q_list)
    a=random.choice(a_list)
    if q_to_a_dict[q]!=a:
        q_a_dataset.append((q,a,-1))



In [18]:
#vectorize
data1=[]
data2=[]
label_list=[]
dataset_list=[]
for data in q_a_dataset:
    q=data[0]
    a=data[1]
    label=data[2]
    v_q=text_dict[q].reshape(-1)
    v_a=text_dict[a].reshape(-1)
    data1.append(v_q)
    data2.append(v_a)
    label_list.append(label)
    dataset_list.append((v_q,v_a,label))


In [19]:
import torch
data1=torch.tensor(data1)
data2=torch.tensor(data2)
labels=torch.tensor(label_list)

#data1,data2,labelsのランダムな90%を学習用、10%を検証用にする
from sklearn.model_selection import train_test_split
train_data1,valid_data1,train_data2,valid_data2,train_labels,valid_labels=train_test_split(data1,data2,labels,test_size=0.1,random_state=42)

In [28]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.nn import functional as F

# ベースとなるネットワークを定義
class BaseNetwork(nn.Module):
    def __init__(self, input_dim=64, hidden_dim=128):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.layer3 = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        #x = (self.layer2(x))
        x = (self.layer3(x))
        return x

# Siamese Networkを定義
class SiameseNetwork(pl.LightningModule):
    def __init__(self, input_dim=64, hidden_dim=128):
        super().__init__()
        self.base_network = BaseNetwork(input_dim, hidden_dim)
        self.loss = nn.CosineEmbeddingLoss()


    def forward(self, input1, input2):
        output1 = self.base_network(input1)
        output2 = self.base_network(input2)
        return output1, output2

    def training_step(self, batch, batch_idx):
        input1, input2, labels = batch
        output1, output2 = self.forward(input1, input2)
        loss = self.loss(output1, output2, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        input1, input2, labels = batch
        output1, output2 = self.forward(input1, input2)
        loss = self.loss(output1, output2, labels)
        self.log('val_loss', loss)
        #print("val_loss:",loss)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())




In [29]:
from pytorch_lightning.callbacks import EarlyStopping
train_dataset = torch.utils.data.TensorDataset(train_data1, train_data2, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32)
val_dataset = torch.utils.data.TensorDataset(valid_data1, valid_data2, valid_labels)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)

model = SiameseNetwork(input_dim=1536,hidden_dim=256)

early_stop_callback = EarlyStopping(monitor='val_loss')
trainer = pl.Trainer(max_epochs=100, callbacks=[early_stop_callback])

trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                | Params
-----------------------------------------------------
0 | base_network | BaseNetwork         | 525 K 
1 | loss         | CosineEmbeddingLoss | 0     
-----------------------------------------------------
525 K     Trainable params
0         Non-trainable params
525 K     Total params
2.100     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [30]:
import matplotlib.pyplot as plt

input1=valid_data1
input2=valid_data2
#input1=train_data1
#input2=train_data2
# モデルを評価モードに設定
model.eval()

# 推論
with torch.no_grad():
    output1, output2 = model(input1, input2)

# コサイン類似度の計算
cosine_similarity = F.cosine_similarity(output1, output2).numpy()

res_df=pd.DataFrame([cosine_similarity,valid_labels.numpy()]).T

res_df.sort_values(1,ascending=False)

,0,1
0,-0.008118,1.0
429,-0.005396,1.0
350,-0.006762,1.0
121,-0.009541,1.0
109,-0.006052,1.0
...,...,...
163,-0.008067,-1.0
161,-0.009267,-1.0
160,-0.154928,-1.0
159,-0.209650,-1.0


0.008620689655172414